In [7]:
import pandas as pd
from UtilsCreateDataFrame import *
import numpy as np
from dataclasses import dataclass, asdict
import matplotlib.pyplot as plt
import numpy_ext as npe
from statsmodels.tsa.stattools import adfuller
from Strategy import *
from Regression import *
import datetime


In [8]:
def loadCreatedDataFrame(path):
    df_ = pd.read_csv(path)
    df_ = df_.set_index(pd.to_datetime(df_.time))
    df_.drop(columns = ["time"], inplace = True)
    return df_
    

path = "BTC_ETH_2021-01.csv.gz"
df_ = loadCreatedDataFrame(path)

In [36]:
"""

with 1minute frequency a day has 1440 observations
let's try this on two week 

"""
index_input = df_.index
x, y = df_.to_numpy()[:,0], df_.to_numpy()[:,1]
window = int(1440/2)
last = None 
rolling = npe.rolling_apply(linearRegression_np, window, x[:last], y[:last], n_jobs = 4)
df_beta = ResultDataFrame(rolling, index_input[:last])
df_beta

,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est
time,,,,,,,
2021-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:00:00+00:00
2021-01-01 00:01:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:01:00+00:00
2021-01-01 00:02:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:02:00+00:00
2021-01-01 00:03:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:03:00+00:00
2021-01-01 00:04:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-01 00:04:00+00:00
...,...,...,...,...,...,...,...
2021-01-31 23:55:00+00:00,0.047019,-234.937598,0.953053,4.580240,-4.462208e-13,0.005607,2021-01-31 23:55:00+00:00
2021-01-31 23:56:00+00:00,0.046977,-233.553110,0.953052,4.571233,-1.932676e-13,0.002388,2021-01-31 23:56:00+00:00
2021-01-31 23:57:00+00:00,0.046937,-232.255770,0.953075,4.561482,6.947529e-14,0.001865,2021-01-31 23:57:00+00:00


In [100]:
df_beta.to_csv("df_beta_ETH_BTC.csv.gz")

In [39]:
"""
TEST 
"""
trading_time = {
    "days":0,
    "hours":12,
    "minutes":0
    }

result_strategy = applyStrategyRolling(
    df_beta = df_beta,
    df_price = df_,
    trading_window = trading_time, 
    thr_pval = 0.1
)

df_trades = strategyDataFrame(result_strategy)
print("\nTotal Return: ", df_trades.ret_.prod(), "\n")
df_trades


Total Return:  68.17545175324764 



,ret_,ret_type,enter_,exit_,beta,intercept,r2,res_std,res_mean,stationarity_pvalue,date_est
0,0.987150,Long,2021-01-01 17:35:00+00:00,2021-01-02 04:51:00+00:00,0.011622,397.478458,0.227892,2.779818,2.971648e-13,0.096245,2021-01-01 16:50:00+00:00
1,1.092282,Long,2021-01-02 08:09:00+00:00,2021-01-02 13:16:00+00:00,0.021739,92.352745,0.447205,2.828115,9.789700e-15,0.096755,2021-01-02 05:04:00+00:00
2,0.832625,Long,2021-01-02 13:53:00+00:00,2021-01-02 17:05:00+00:00,0.021739,92.352745,0.447205,2.828115,9.789700e-15,0.096755,2021-01-02 05:04:00+00:00
3,1.042231,Long,2021-01-02 18:57:00+00:00,2021-01-02 20:46:00+00:00,0.016905,230.781227,0.928890,4.378014,8.605462e-14,0.097838,2021-01-02 17:08:00+00:00
4,1.010956,Short,2021-01-02 21:01:00+00:00,2021-01-02 21:05:00+00:00,0.016905,230.781227,0.928890,4.378014,8.605462e-14,0.097838,2021-01-02 17:08:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...
68,0.983887,Short,2021-01-31 02:34:00+00:00,2021-01-31 09:39:00+00:00,0.047724,-266.055864,0.560419,9.320632,-8.021238e-14,0.090393,2021-01-30 21:38:00+00:00
69,1.251946,Short,2021-01-31 11:29:00+00:00,2021-01-31 12:38:00+00:00,0.039850,8.450003,0.935620,3.210098,-3.000069e-14,0.052993,2021-01-31 09:40:00+00:00
70,18.015478,Long,2021-01-31 15:12:00+00:00,2021-01-31 18:14:00+00:00,0.039850,8.450003,0.935620,3.210098,-3.000069e-14,0.052993,2021-01-31 09:40:00+00:00
71,-3.340686,Long,2021-01-31 18:45:00+00:00,2021-01-31 21:41:00+00:00,0.039850,8.450003,0.935620,3.210098,-3.000069e-14,0.052993,2021-01-31 09:40:00+00:00


In [46]:
row = df_trades.iloc[70]
print(row)

ret_                                   18.015478
ret_type                                    Long
enter_                 2021-01-31 15:12:00+00:00
exit_                  2021-01-31 18:14:00+00:00
beta                                     0.03985
intercept                               8.450003
r2                                       0.93562
res_std                                 3.210098
res_mean                                    -0.0
stationarity_pvalue                     0.052993
date_est               2021-01-31 09:40:00+00:00
Name: 70, dtype: object


In [49]:
p0 = df_.ETH.loc[row.enter_] - row.beta * df_.BTC.loc[row.enter_]
print(df_.ETH.loc[row.enter_]  )
print(df_.BTC.loc[row.enter_]   )
print(p0)

1327.09
33294.04
0.31426965259242934


In [99]:
p1 = df_.ETH.loc[row.exit_] - row.beta * df_.BTC.loc[row.exit_]
print(df_.ETH.loc[row.exit_]  )
print(df_.BTC.loc[row.exit_]   )
print(p1)
(p1-p0)/p0

1302.3
32472.62
8.258059857144872


25.27698789566106

In [87]:
def return_(name, enter, exit):
    return  (df_.loc[exit, name]-df_.loc[enter, name])

In [88]:
eth = return_("ETH", row.enter_, row.exit_)
eth

-24.789999999999964

In [89]:
btc = return_("BTC", row.enter_, row.exit_)
btc

-821.4200000000019

In [97]:
gain = eth - row.beta*btc
gain

7.943790204552258